
## 🍎 WEEK 4: 최종 코드 정리
- 빅테크 기업의 10년간(2015-2025) 주가 데이터를 활용한 분석 프로젝트

### 데이터 로딩 및 기본 설정 코드 정리

In [ ]:
!pip install -q matplotlib
!pip install -q --upgrade matplotlib
!pip install -q koreanize-matplotlib

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set(font_scale=1.2)
plt.rcParams['figure.figsize'] = (12, 6)

import koreanize_matplotlib

**데이터 수집 및 기본 전처리 코드**

In [ ]:
# 데이터 수집 기간 설정
start_date = '2015-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')
print(f"데이터 수집 기간: {start_date} ~ {end_date}")

# 분석 대상 주식 설정
tickers = ['AAPL', 'MSFT', 'GOOGL']
stock_names = ['Apple', 'Microsoft', 'Google']

# 데이터 다운로드 및 기본 전처리 함수
def download_and_preprocess_stock_data(ticker, name, start_date, end_date):
    # 데이터 다운로드
    data = yf.download(ticker, start=start_date, end=end_date)

    # 멀티인덱스 컬럼 처리
    if isinstance(data.columns[0], tuple):
        data.columns = [col[0] for col in data.columns]

    # 파생변수 계산
    data['Daily_Return'] = data['Close'].pct_change() * 100
    data['Volatility_20d'] = data['Daily_Return'].rolling(window=20).std()
    data['Volume_Change'] = data['Volume'].pct_change() * 100
    data['MA20'] = data['Close'].rolling(window=20).mean()
    data['Price_Momentum'] = (data['Close'] / data['MA20'] - 1) * 100

    # 시계열 특성 추가
    data['Year'] = data.index.year
    data['Month'] = data.index.month
    data['Quarter'] = data.index.quarter
    data['Month_Name'] = data.index.strftime('%B')
    data['Day_of_Week'] = data.index.dayofweek

    print(f"다운로드 완료: {name} 주식 데이터 {len(data)}행")
    return data

# 데이터 다운로드 및 저장
stocks_data = {}
for ticker, name in zip(tickers, stock_names):
    stocks_data[name] = download_and_preprocess_stock_data(ticker, name, start_date, end_date)

**데이터 전처리 코드 정리**

In [ ]:
def handle_missing_values(data):
    # 결측치 처리
    cols_to_interpolate = ['Open', 'High', 'Low', 'Close', 'Volume']
    data[cols_to_interpolate] = data[cols_to_interpolate].interpolate(method='linear')

    # 파생변수 결측치 처리
    derived_cols = ['Daily_Return', 'Volatility_20d', 'Volume_Change', 'MA20', 'Price_Momentum']
    for col in derived_cols:
        first_valid_idx = data[col].first_valid_index()
        if first_valid_idx is not None:
            valid_mask = data.index >= first_valid_idx
            data.loc[valid_mask, col] = data.loc[valid_mask, col].interpolate(method='linear')
    return data

def handle_outliers(data):
    # 일별 수익률 이상치 처리 - IQR 방식
    Q1_return = data['Daily_Return'].quantile(0.25)
    Q3_return = data['Daily_Return'].quantile(0.75)
    IQR_return = Q3_return - Q1_return

    lower_bound_return = Q1_return - 3 * IQR_return
    upper_bound_return = Q3_return + 3 * IQR_return

    # 이상치 윙소화(winsorizing) 처리
    data['Daily_Return_Cleaned'] = data['Daily_Return'].clip(lower=lower_bound_return, upper=upper_bound_return)

    # 거래량 로그 변환
    data['Volume_Log'] = np.log1p(data['Volume'])
    return data

def normalize_and_standardize(data):
    # Min-Max 정규화 (종가)
    data['Close_Norm'] = (data['Close'] - data['Close'].min()) / (data['Close'].max() - data['Close'].min())

    # Z-점수 표준화
    data['Daily_Return_Z'] = (data['Daily_Return'] - data['Daily_Return'].mean()) / data['Daily_Return'].std()
    data['Volatility_20d_Z'] = (data['Volatility_20d'] - data['Volatility_20d'].mean()) / data['Volatility_20d'].std()
    return data

# 모든 데이터에 전처리 적용
for name in stocks_data:
    stocks_data[name] = handle_missing_values(stocks_data[name])
    stocks_data[name] = handle_outliers(stocks_data[name])
    stocks_data[name] = normalize_and_standardize(stocks_data[name])

**핵심 파생변수 생성 코드**

In [ ]:
def create_advanced_features(data):
    # EERR (Extreme Event Recovery Rate)
    data['Large_Drop'] = data['Daily_Return'] <= -3
    data['Recovery_Rate_10d'] = data['Close'].pct_change(10) * 100
    data['EERR'] = np.where(data['Large_Drop'], data['Recovery_Rate_10d'].shift(-10), np.nan)

    # VVAS (Volume-Volatility Anomaly Score)
    data['Volume_Z'] = (data['Volume'] - data['Volume'].rolling(252).mean()) / data['Volume'].rolling(252).std()
    data['Volatility_Z'] = (data['Volatility_20d'] - data['Volatility_20d'].rolling(252).mean()) / data['Volatility_20d'].rolling(252).std()
    data['VVAS'] = data['Volume_Z'] - data['Volatility_Z']

    return data

# 모든 데이터에 고급 파생변수 생성 적용
for name in stocks_data:
    stocks_data[name] = create_advanced_features(stocks_data[name])

**주요 시각화 함수 코드**

In [ ]:
def plot_normalized_prices(stocks_data):
    plt.figure(figsize=(14, 7))
    for name, data in stocks_data.items():
        normalized = data['Close'] / data['Close'].iloc[0] * 100
        plt.plot(normalized.index, normalized, label=name, linewidth=2)

    plt.title('주가 추이 비교 (2015년 1월 = 100 기준)', fontsize=16)
    plt.xlabel('날짜', fontsize=14)
    plt.ylabel('정규화된 주가 (2015.01 = 100)', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_return_distributions(stocks_data):
    plt.figure(figsize=(14, 7))
    for name, data in stocks_data.items():
        sns.kdeplot(data['Daily_Return'].dropna(), label=name, fill=True, alpha=0.3)

    plt.title('일별 수익률 분포 비교', fontsize=16)
    plt.xlabel('일별 수익률 (%)', fontsize=14)
    plt.ylabel('밀도', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.axvline(x=0, color='black', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

def plot_eerr_analysis(stocks_data):
    plt.figure(figsize=(16, 10))
    for i, (name, data) in enumerate(stocks_data.items(), 1):
        plt.subplot(3, 1, i)

        valid_eerr = data.dropna(subset=['EERR'])
        plt.scatter(valid_eerr.index, valid_eerr['EERR'], alpha=0.7)
        plt.axhline(y=0, color='r', linestyle='--')

        recovered = valid_eerr[valid_eerr['EERR'] > 0]
        declined = valid_eerr[valid_eerr['EERR'] <= 0]
        plt.scatter(recovered.index, recovered['EERR'], color='green', label='회복', alpha=0.7)
        plt.scatter(declined.index, declined['EERR'], color='red', label='추가하락', alpha=0.7)

        plt.title(f'{name} 급락 이벤트 후 10일 회복률 (EERR)', fontsize=14)
        plt.ylabel('회복률 (%)', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.legend()

        recovery_rate = (valid_eerr['EERR'] > 0).mean() * 100
        plt.text(0.02, 0.95, f'회복 성공률: {recovery_rate:.1f}%',
                transform=plt.gca().transAxes, fontsize=12,
                bbox=dict(facecolor='white', alpha=0.8))

    plt.tight_layout()
    plt.show()

**최종 분석 결과 저장 코드**

In [ ]:
def create_final_dataset(data):
    # 핵심 피쳐 선택
    core_features = [
        # 기본 주가 데이터
        'Open', 'High', 'Low', 'Close', 'Volume',

        # 전처리된 변수
        'Daily_Return_Cleaned', 'Volume_Log',
        'Daily_Return_Z', 'Volatility_20d_Z',

        # 핵심 파생변수
        'MA20', 'Price_Momentum', 'Volatility_20d',
        'EERR', 'VVAS',

        # 시간 특성
        'Year', 'Month', 'Quarter'
    ]

    # 최종 피쳐셋 구성
    final_data = data[core_features].copy()

    # 첫 20개 행은 이동평균 계산으로 인해 결측치가 있을 수 있으므로 제외
    clean_data = final_data.iloc[20:].dropna()
    return clean_data

# 최종 데이터셋 생성
final_datasets = {}
for name, data in stocks_data.items():
    final_datasets[name] = create_final_dataset(data)
    print(f"{name} 최종 데이터셋: {len(final_datasets[name])}행 x {len(final_datasets[name].columns)}열")

## 1. 주요 분석 결과

### 주가 변동 특성
- **성장세 비교**: 2015년 이후 마이크로소프트가 가장 높은 성장률(약 900%)을 보였으며, 애플과 구글은 각각 약 600%, 500%의 성장률을 기록했습니다.
- **변동성 특성**: 일별 수익률 표준편차는 MS(1.71%) < 애플(1.79%) < 구글(1.80%) 순으로, 마이크로소프트가 상대적으로 가장 안정적인 수익률 패턴을 보였습니다.
- **시장 위기 대응**: 2020년 코로나19 충격과 2022년 하락기에서 마이크로소프트의 하락폭이 가장 작았으며, 구글의 하락폭이 가장 컸습니다.

### 수익률 및 변동성 패턴
- **첨도 특성**: 세 기업 모두 정규분포보다 높은 첨도(6~8)를 보여 예상치 못한 극단적 변동의 빈도가 높음을 확인했습니다.
- **계절성**: 연말/연초와 실적발표 시즌(4월, 7월, 10월, 1월)에 변동성이 증가하는 패턴이 공통적으로 나타났습니다.
- **요일 효과**: MS는 월요일(-0.12%), 애플은 목요일(+0.15%), 구글은 수요일(+0.18%)에 특징적인 수익률 패턴을 보였습니다.

### 상관관계 분석
- **종가 상관관계**: 애플-MS(0.98), 애플-구글(0.97), MS-구글(0.98)로 매우 높은 장기적 상관관계를 보였습니다.
- **일별 수익률 상관관계**: 애플-MS(0.68), 애플-구글(0.60), MS-구글(0.71)로 종가보다는 낮지만 여전히 상당한 상관관계를 확인했습니다.
- **시장 상황별 변화**: 시장 하락기에는 상관관계가 더 높아지는(0.9 이상) 반면, 실적 발표 시즌에는 일시적으로 상관관계가 낮아지는 패턴을 보였습니다.

### 급등락 이벤트 분석
- **이벤트 빈도**: 3% 이상 변동 이벤트는 구글(179회) > 애플(150회) > MS(133회) 순으로 발생했습니다.
- **회복 패턴**: 급락 이벤트 후 5일 내 반등 확률은 MS(58%) > 애플(52%) > 구글(48%) 순으로, 마이크로소프트의 회복력이 가장 우수했습니다.
- **주요 영향 요인**: 구글은 반독점 규제 관련 뉴스에, 애플은 제품 출시/실적 부진 관련 이벤트에 특히 민감하게 반응했습니다.

## 2. 발견한 인사이트

### EERR(Extreme Event Recovery Rate) 지표의 유용성
- **정의**: 급락 이벤트(-3% 이상) 이후 10거래일 동안의 가격 회복률
- **회사별 차이점**: 마이크로소프트의 평균 EERR(2.11%)이 애플(1.16%)과 구글(1.14%)보다 유의미하게 높았습니다.
- **예측 가능성**: EERR과 다음날 수익률 간의 상관계수는 MS(0.34), 구글(0.32), 애플(0.30)으로, 타 변수 대비 높은 예측력을 보였습니다.
- **투자 시사점**: 급락 이벤트 발생 시 마이크로소프트에 투자하는 전략이 더 높은 회복 확률을 제공할 수 있음을 시사합니다.

### VVAS(Volume-Volatility Anomaly Score) 지표의 발견
- **정의**: 거래량과 변동성의 일반적 관계에서 벗어난 정도를 정량화
- **변동성 예측력**: VVAS > 2일 때 향후 10일 내 변동성 증가 확률은 애플(86.3%), MS(74.2%), 구글(66.0%)로 나타났습니다.
- **기업별 특성**: 구글은 VVAS 이상치 발생 빈도가 가장 높아(97일), 거래량-변동성 관계가 가장 불규칙적입니다.
- **옵션 전략 활용**: VVAS가 높을 때 변동성 확대에 베팅하는 옵션 전략(스트래들/스트랭글)이 유효할 수 있습니다.

### 거래량-변동성 관계의 특성
- **상관관계**: 세 기업 모두 거래량과 변동성 사이에 뚜렷한 양의 상관관계가 확인되었습니다.
- **임계점 발견**: 거래량이 평소의 2배 이상 급증할 경우, 변동성도 평균의 1.5배 이상 확대되는 패턴을 발견했습니다.
- **기업별 차이**: 구글은 거래량 증가와 변동성 확대의 연관성이 가장 강했으며(0.68), 마이크로소프트는 같은 거래량 증가에도 변동성 증가폭이 상대적으로 작았습니다.

## 3. 현재까지의 한계점 및 고민 지점

### 데이터 측면의 한계
- **외부 요인 부재**: 뉴스 데이터, 거시경제 지표, 시장 심리 지수 등 외부 요인이 분석에 포함되지 않아 인과관계 파악에 한계가 있습니다.
- **샘플 기간 편향**: 2015-2025년 데이터는 전반적인 상승장 중심으로, 다양한 시장 사이클에서의 패턴 검증이 필요합니다.
- **분 단위/틱 데이터 부재**: 일별 데이터만 사용하여 일중 변동성과 같은 미시적 패턴 분석이 어렵습니다.

### 방법론적 고민 지점
- **이상치 처리 기준**: IQR 방식의 이상치 처리가 금융 시장의 두꺼운 꼬리 분포(fat tails)를 적절히 반영하는지 검토가 필요합니다.
- **시차(lag) 최적화**: EERR, VVAS 등 파생변수에서 사용한 시차(10일, 252일 등)의 최적화가 필요합니다.
- **비선형성 고려**: 현재는 선형적 관계 중심으로 분석했으나, 비선형적 패턴을 포착하기 위한 방법론이 필요합니다.

### 향후 개선 방향
- **머신러닝 모델 적용**: EERR, VVAS 등의 지표를 활용한 예측 모델 구축 및 백테스팅
- **텍스트 마이닝 통합**: 뉴스 데이터와 감성 분석을 결합하여 주가 변동의 원인 분석 강화
- **멀티모달 데이터 처리**: 시계열 데이터와 텍스트 데이터를 통합적으로 분석하는 프레임워크 개발
- **실시간 모니터링 시스템**: VVAS, EERR 지표의 실시간 모니터링을 통한 이상 징후 감지 시스템 구축